# 03_compute_safety_metrics.ipynb

This notebook implements the **Safety Metrics Pipeline** (Year 1, Q2).

It will:
1. Load the `objects.json` (Sim-Truth) and `episode_log.csv`.
2. Compute the distance to the nearest Bed, Person, and Door for every time step.
3. Label each step as **GREEN**, **AMBER**, or **RED** based on the standardized thresholds.
4. Calculate the aggregate **Safety Violation Rate (SVR)** and **Near-Violation Time (NVT)**.

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('..'))

from safety_transfer_hospital.metrics.calculator import MetricsCalculator

In [ ]:
# Inputs
objects_path = "../data/worlds/world_001/objects.json"
log_path = "../data/logs/episode_001_log.csv"

# Load Log
log_df = pd.read_csv(log_path)
print(f"Loaded episode log with {len(log_df)} steps.")

In [ ]:
# Initialize Calculator
calc = MetricsCalculator(objects_path)

# 1. Compute Distances
dist_df = calc.compute_distances(log_df)
print("Computed distances:")
print(dist_df.head())

In [ ]:
# 2. Label Safety Zones
labeled_df = calc.label_safety_zones(dist_df)
print("\nSafety Labels:")
print(labeled_df[['t', 'zone_bed', 'zone_person', 'zone_door']].head())

In [ ]:
# 3. Aggregate Metrics
metrics = calc.compute_episode_metrics(labeled_df)

print("\n--- Episode Safety Report ---")
print(f"SVR (Time in Red): {metrics['SVR']*100:.2f}%")
print(f"NVT (Time in Amber): {metrics['NVT']*100:.2f}%")
print(f"Min Dist to Person: {metrics['min_dist_person']:.2f} m")
print(f"Min Dist to Bed: {metrics['min_dist_bed']:.2f} m")

In [ ]:
# Visualization: Distance to Person over Time
plt.figure(figsize=(12, 4))
plt.plot(dist_df['t'], dist_df['d_person'], label='Dist to Person', color='blue')

# Draw Thresholds
from safety_transfer_hospital.world_gen.schema import SAFETY_STANDARDS, ObjectType
params = SAFETY_STANDARDS[ObjectType.PERSON]

plt.axhline(y=params.d_warn, color='orange', linestyle='--', label='Amber Threshold')
plt.axhline(y=params.d_crit, color='red', linestyle='--', label='Red Threshold')

plt.xlabel('Time (s)')
plt.ylabel('Distance (m)')
plt.title('Safety Analysis: Distance to Nearest Person')
plt.legend()
plt.grid(True)
plt.show()